# Main Notebook - Hackathon Project 2/2022

## Part I. Data Inspection, Pre-Processing and Cleaning

In [37]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

import seaborn as sns
sns.set_theme(style='darkgrid')

import sklearn.metrics
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    RandomizedSearchCV,
    cross_val_score
)
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
    BaggingClassifier,
)
from xgboost import XGBClassifier

import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

import warnings

warnings.filterwarnings("ignore")

In [38]:
#import training and test data into pandas dataframe and make copies
raw_train_data = pd.read_csv('Train_set.csv')
raw_test_data  = pd.read_csv('Test_set.csv')

train_data     = raw_train_data.copy(deep=True)
test_data     = raw_test_data.copy(deep=True)

In [39]:
train_data.head()

,ID,loan_amnt,loan_term,interest_rate,loan_grade,loan_subgrade,job_experience,home_ownership,annual_income,income_verification_status,...,delinq_2yrs,public_records,revolving_balance,total_acc,interest_receive,application_type,last_week_pay,total_current_balance,total_revolving_limit,default
0,72199369,9000,3 years,9.17,B,B2,<5 Years,OWN,85000.0,Not Verified,...,0.0,0.0,39519,20.0,59.60,INDIVIDUAL,4.0,95493.0,84100.0,0
1,14257956,18000,3 years,13.65,C,C1,<5 Years,OWN,64000.0,Verified,...,0.0,1.0,9783,24.0,3348.25,INDIVIDUAL,95.0,185433.0,13500.0,0
2,66216451,16000,3 years,7.26,A,A4,<5 Years,MORTGAGE,150000.0,Source Verified,...,2.0,0.0,13641,27.0,276.69,INDIVIDUAL,13.0,180519.0,19300.0,0
3,46974169,25000,3 years,13.99,C,C4,NaN,MORTGAGE,59800.0,Verified,...,0.0,0.0,35020,35.0,1106.72,INDIVIDUAL,17.0,183208.0,55400.0,0
4,46725961,17000,3 years,6.39,A,A2,10+ years,MORTGAGE,72000.0,Source Verified,...,0.0,0.0,23990,26.0,725.29,INDIVIDUAL,39.0,23990.0,81300.0,0


In [40]:
train_data.describe(include='all')

,ID,loan_amnt,loan_term,interest_rate,loan_grade,loan_subgrade,job_experience,home_ownership,annual_income,income_verification_status,...,delinq_2yrs,public_records,revolving_balance,total_acc,interest_receive,application_type,last_week_pay,total_current_balance,total_revolving_limit,default
count,9.317400e+04,93174.000000,93174,93174.000000,93174,93174,88472,93174,9.317300e+04,93174,...,93172.000000,93172.000000,9.317400e+04,93172.000000,93174.000000,93174,91250.000000,8.578800e+04,8.578800e+04,93174.000000
unique,NaN,NaN,2,NaN,7,35,3,5,NaN,3,...,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN
top,NaN,NaN,3 years,NaN,B,B3,<5 Years,MORTGAGE,NaN,Source Verified,...,NaN,NaN,NaN,NaN,NaN,INDIVIDUAL,NaN,NaN,NaN,NaN
freq,NaN,NaN,65211,NaN,26865,5879,40610,46445,NaN,34487,...,NaN,NaN,NaN,NaN,NaN,93118,NaN,NaN,NaN,NaN
mean,3.505021e+07,14733.861378,NaN,13.233433,NaN,NaN,NaN,NaN,7.502826e+04,NaN,...,0.316994,0.196121,1.685447e+04,25.248905,1747.263530,NaN,58.154882,1.392529e+05,3.208590e+04,0.237502
std,2.414926e+07,8428.184858,NaN,4.368728,NaN,NaN,NaN,NaN,6.945478e+04,NaN,...,0.880859,0.581188,2.368907e+04,11.855061,2088.236141,NaN,44.326560,1.576868e+05,4.705251e+04,0.425555
min,7.073500e+04,500.000000,NaN,5.320000,NaN,NaN,NaN,NaN,1.200000e+03,NaN,...,0.000000,0.000000,0.000000e+00,1.000000,0.000000,NaN,0.000000,0.000000e+00,0.000000e+00,0.000000
25%,1.085983e+07,8000.000000,NaN,9.990000,NaN,NaN,NaN,NaN,4.500000e+04,NaN,...,0.000000,0.000000,6.433000e+03,17.000000,439.880000,NaN,22.000000,2.964200e+04,1.400000e+04,0.000000
50%,3.710751e+07,13000.000000,NaN,12.990000,NaN,NaN,NaN,NaN,6.400000e+04,NaN,...,0.000000,0.000000,1.185600e+04,24.000000,1070.755000,NaN,48.000000,7.936350e+04,2.370000e+04,0.000000
75%,5.859895e+07,20000.000000,NaN,16.200000,NaN,NaN,NaN,NaN,9.000000e+04,NaN,...,0.000000,0.000000,2.074500e+04,32.000000,2219.612500,NaN,83.000000,2.071600e+05,3.970000e+04,0.000000


In [41]:
#check for duplicate values
train_data.duplicated().sum()

0

In [42]:
test_data.duplicated().sum()

0

In [43]:
#check for missing values
train_data.isnull().sum()

ID                               0
loan_amnt                        0
loan_term                        0
interest_rate                    0
loan_grade                       0
loan_subgrade                    0
job_experience                4702
home_ownership                   0
annual_income                    1
income_verification_status       0
loan_purpose                     0
state_code                       0
debt_to_income                   0
delinq_2yrs                      2
public_records                   2
revolving_balance                0
total_acc                        2
interest_receive                 0
application_type                 0
last_week_pay                 1924
total_current_balance         7386
total_revolving_limit         7386
default                          0
dtype: int64

In [44]:
test_data.isnull().sum()

ID                               0
loan_amnt                        0
loan_term                        0
interest_rate                    0
loan_grade                       0
loan_subgrade                    0
job_experience                2089
home_ownership                   0
annual_income                    0
income_verification_status       0
loan_purpose                     0
state_code                       0
debt_to_income                   0
delinq_2yrs                      1
public_records                   1
revolving_balance                0
total_acc                        1
interest_receive                 0
application_type                 0
last_week_pay                  806
total_current_balance         3230
total_revolving_limit         3230
dtype: int64

In [46]:
#unique ID is for record keeping and unique to each entry - check to make sure no IDs
#are included in test set and training set by mistake
print(all((np.where(train_data["ID"].isin(test_data["ID"])), True, False)))

False


In [51]:
#no user IDs are inadvertently included in both data sets. We can safely drop ID as an identifier because it
#is completely unique
train_data.drop("ID", axis=1, inplace=True)
test_data.drop("ID", axis=1, inplace=True)

train_data.head()

,loan_amnt,loan_term,interest_rate,loan_grade,loan_subgrade,job_experience,home_ownership,annual_income,income_verification_status,loan_purpose,...,delinq_2yrs,public_records,revolving_balance,total_acc,interest_receive,application_type,last_week_pay,total_current_balance,total_revolving_limit,default
0,9000,3 years,9.17,B,B2,<5 Years,OWN,85000.0,Not Verified,debt_consolidation,...,0.0,0.0,39519,20.0,59.60,INDIVIDUAL,4.0,95493.0,84100.0,0
1,18000,3 years,13.65,C,C1,<5 Years,OWN,64000.0,Verified,debt_consolidation,...,0.0,1.0,9783,24.0,3348.25,INDIVIDUAL,95.0,185433.0,13500.0,0
2,16000,3 years,7.26,A,A4,<5 Years,MORTGAGE,150000.0,Source Verified,debt_consolidation,...,2.0,0.0,13641,27.0,276.69,INDIVIDUAL,13.0,180519.0,19300.0,0
3,25000,3 years,13.99,C,C4,NaN,MORTGAGE,59800.0,Verified,debt_consolidation,...,0.0,0.0,35020,35.0,1106.72,INDIVIDUAL,17.0,183208.0,55400.0,0
4,17000,3 years,6.39,A,A2,10+ years,MORTGAGE,72000.0,Source Verified,credit_card,...,0.0,0.0,23990,26.0,725.29,INDIVIDUAL,39.0,23990.0,81300.0,0


In [52]:
test_data.head()

,loan_amnt,loan_term,interest_rate,loan_grade,loan_subgrade,job_experience,home_ownership,annual_income,income_verification_status,loan_purpose,...,debt_to_income,delinq_2yrs,public_records,revolving_balance,total_acc,interest_receive,application_type,last_week_pay,total_current_balance,total_revolving_limit
0,12000,3 years,15.31,C,C2,<5 Years,MORTGAGE,73400.0,Not Verified,debt_consolidation,...,14.62,0.0,0.0,22156,30.0,2290.24,INDIVIDUAL,87.0,128098.0,25800.0
1,12000,3 years,7.26,A,A4,10+ years,MORTGAGE,105000.0,Not Verified,credit_card,...,11.38,0.0,0.0,7592,14.0,202.68,INDIVIDUAL,13.0,269396.0,23600.0
2,15000,3 years,14.33,C,C1,6-10 years,MORTGAGE,50000.0,Verified,debt_consolidation,...,28.15,0.0,1.0,17983,19.0,1166.24,INDIVIDUAL,30.0,220135.0,34100.0
3,12000,3 years,9.99,B,B3,6-10 years,RENT,37000.0,Source Verified,credit_card,...,34.32,0.0,0.0,12262,18.0,635.06,INDIVIDUAL,35.0,39436.0,21700.0
4,20150,3 years,11.53,B,B5,<5 Years,RENT,75000.0,Source Verified,debt_consolidation,...,26.74,1.0,0.0,8251,11.0,1232.84,INDIVIDUAL,31.0,52764.0,12000.0


In [53]:
#let's check each of the columns to learn more about the nature of the missing values and the data types
train_data["loan_amnt"].nunique()

1310

In [54]:
#checking to make sure there are no zero or negative loan amounts
print(all(np.where(train_data["loan_amnt"] <= 0, True, False)))

False


In [55]:
print(all(np.where(test_data["loan_amnt"] <= 0, True, False)))

False


In [62]:
#repeat for interest rate
print(all(np.where(train_data["interest_rate"] <= 0, True, False)))

False


In [65]:
print(all(np.where(test_data["interest_rate"] <= 0, True, False)))

False


In [66]:
#repeat for annual income
print(all(np.where(train_data["annual_income"] <= 0, True, False)))

False


In [67]:
print(all(np.where(test_data["annual_income"] <= 0, True, False)))

False


In [70]:
#checking debt-to-income ratio
print(all(np.where(train_data["debt_to_income"] <= 0, True, False)))

False


In [71]:
print(all(np.where(test_data["debt_to_income"] <= 0, True, False)))

False


In [56]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93174 entries, 0 to 93173
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   93174 non-null  int64  
 1   loan_term                   93174 non-null  object 
 2   interest_rate               93174 non-null  float64
 3   loan_grade                  93174 non-null  object 
 4   loan_subgrade               93174 non-null  object 
 5   job_experience              88472 non-null  object 
 6   home_ownership              93174 non-null  object 
 7   annual_income               93173 non-null  float64
 8   income_verification_status  93174 non-null  object 
 9   loan_purpose                93174 non-null  object 
 10  state_code                  93174 non-null  object 
 11  debt_to_income              93174 non-null  float64
 12  delinq_2yrs                 93172 non-null  float64
 13  public_records              931

In [60]:
train_data["loan_term"].value_counts()

3 years    65211
5 years    27963
Name: loan_term, dtype: int64

In [61]:
test_data["loan_term"].value_counts()

3 years    28110
5 years    11823
Name: loan_term, dtype: int64

## Part II. Exploratory Data Analysis

## Part III. Feature Engineering and Encoding

## Part IV. Building the Models on Training Data

## Part V. Model Performance on Testing Data